## Setup

In [20]:
import numpy as np
import pandas as pd
import torch
import nltk
import zipfile
import fasttext

from pathlib import Path
from InferSent.models import InferSent
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

nltk.download('punkt')
nltk.download('stopwords')
set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daddy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Daddy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [2]:
# clone or pull InferSent
if not Path("InferSent").is_dir():
    !git clone https://github.com/facebookresearch/InferSent.git
else:
    %cd InferSent
    !git pull
    %cd ..

# download and preprocess SNLI/MultiNLI datasets for fastText
if not Path("InferSent/dataset/fastText").exists():
    %cd Infersent/dataset
    %mkdir fastText
    %cd ..
    %cd ..
    
if not Path("InferSent/dataset/fastText/crawl-300d-2M.vec.zip").is_file():
    !curl -Lo dataset/fastText/crawl-300d-2M.vec.zip https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M.vec.zip

if not Path("InferSent/encoder/infersent2.pkl").is_file():
    !curl -Lo InferSent/encoder/infersent2.pkl https://s3.amazonaws.com/senteval/infersent/infersent2.pkl

D:\System\Gamenomicron\AI\MLND\project\InferSent
Already up to date.
D:\System\Gamenomicron\AI\MLND\project


In [3]:
if not Path("InferSent/dataset/fastText/crawl-300d-2M.vec/crawl-300d-2M.vec").is_file():
    zipfile.ZipFile("InferSent/dataset/fastText/crawl-300d-2M.vec.zip").extractall(".").close()

In [15]:
# Load pretrained InferSent model

infersent_version = 2
INFERSENT_PATH = "InferSent/encoder/infersent%s.pkl" % infersent_version
infersent_params = {
    'bsize': 64,
    'word_emb_dim': 300,
    'enc_lstm_dim': 2048,
    'pool_type': 'max',
    'dpout_model': 0.0,
    'version': infersent_version
}
infersent = InferSent(infersent_params)
infersent.load_state_dict(torch.load(INFERSENT_PATH))

In [16]:
# setting the word embeddings path to fastText
infersent.set_w2v_path("InferSent/dataset/fastText/crawl-300d-2M.vec/crawl-300d-2M.vec")

In [17]:
# Load embeddings of K most frequent words

infersent.build_vocab_k_words(K=100000)

Vocab size : 100000


## Loading the data

In [27]:
raw_data = pd.read_csv('data/Combined_News_DJIA.csv')

raw_data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [28]:
raw_data_dropped = raw_data.dropna()

## Preprocessing

In [29]:
# stop word removal function

def generate_stop_word_free_dataset(dataframe):
    df = dataframe.copy()
    # iterate over every column
    for column in df.columns:
        # convert column values into an array where each row is an element in the array
        sentences = df[column].tolist()
        # remove stop words
        if isinstance(sentences[0], str): # doesn't seem to be working properly
            stop_wordless_sentences = []
            for sentence in sentences:
                split_sentence = sentence.split()
                new_words = [word for word in split_sentence if word not in stopwords.words('english')]
                # reconstruct the sentence
                new_sentence = " ".join(new_words)
                stop_wordless_sentences.append(new_sentence)
            # recolumnize into dataframe
            df[column] = stop_wordless_sentences
    return(df)

In [30]:
# creating our bearish holdout set for later testing

raw_data_dropped = raw_data_dropped.set_index(['Date']) # sets index to the date column so we can split by date ranges
bearish_raw_data = raw_data_dropped.loc['2008-08-08':'2009-03-31']
bullish_raw_data = raw_data_dropped.loc['2009-04-01':'2016-07-01']

# resets all the indexes just because I feel like it
raw_data_dropped = raw_data_dropped.reset_index()
bearish_raw_data = bearish_raw_data.reset_index()
bullish_raw_data = bullish_raw_data.reset_index()
bullish_raw_data_labels = bullish_raw_data.pop('Label')
bearish_raw_data_labels = bearish_raw_data.pop('Label')

# generate stop wordless datasets

bullish_stop_words_free_data = generate_stop_word_free_dataset(bullish_raw_data)
bearish_stop_words_free_data = generate_stop_word_free_dataset(bearish_raw_data)

In [53]:
def convert_sentences_to_vector(dataframe):
    df = dataframe.copy()
    # iterate over every column
    for column in df.columns:
        # convert column values into an array where each row is an element in the array
        sentences = df[column].tolist()
        sentences_vectors = infersent.encode(sentences)
        for idx, sentence_vectors in enumerate(sentences_vectors):
            df.at[idx, column] = sentence_vectors
    return df

In [32]:
# preprocessing_pipeline = Pipeline([
#  ('vectorization', FunctionTransformer(convert_sentences_to_vector))   
# ])

In [55]:
data = convert_sentences_to_vector(bullish_raw_data)

In [56]:
# splitting data before preprocessing so as to avoid introducing unseen information from the test set into the training

def split_data(x_data, y_data, test_percent_size):
    return train_test_split(x_data, y_data, test_size=test_percent_size)

x_train, x_test, y_train, y_test = split_data(data, bullish_raw_data_labels, 0.2)

In [57]:
model = SVC()
model.fit(x_train, y_train)

ValueError: setting an array element with a sequence.